In [1]:
from itertools import product, combinations, cycle
from functools import reduce
from math import prod

In [4]:
fname = 'example.txt'
# fname = 'input.txt'
# fname = 'example2.txt'
        
with open(fname) as f:
    lines = f.readlines()

# [(on, [int(num) for num in rang.replace('x=', '').replace('y=', '').replace('z', '').replace('=', '').split('..') for rang in xyz.split(',')]) for on, xyz in [line.strip().split(' ') for line in lines]]

onoffs, ranges = [], []
# for line in lines[0:20]:
for line in lines[:-2]:
# for line in lines:
    onoff, xyz = line.strip().split(' ')
    ranges_letters = xyz.split(',')
    range_3d = tuple([tuple(int(val) for val in range_letter.split('=')[1].split('..')) for range_letter in ranges_letters])
    onoffs.append(onoff)
    ranges.append(range_3d)

instructions = list(zip(onoffs, ranges))
# instructions

In [5]:
instructions

[('on', ((-20, 26), (-36, 17), (-47, 7))),
 ('on', ((-20, 33), (-21, 23), (-26, 28))),
 ('on', ((-22, 28), (-29, 23), (-38, 16))),
 ('on', ((-46, 7), (-6, 46), (-50, -1))),
 ('on', ((-49, 1), (-3, 46), (-24, 28))),
 ('on', ((2, 47), (-22, 22), (-23, 27))),
 ('on', ((-27, 23), (-28, 26), (-21, 29))),
 ('on', ((-39, 5), (-6, 47), (-3, 44))),
 ('on', ((-30, 21), (-8, 43), (-13, 34))),
 ('on', ((-22, 26), (-27, 20), (-29, 19))),
 ('off', ((-48, -32), (26, 41), (-47, -37))),
 ('on', ((-12, 35), (6, 50), (-50, -2))),
 ('off', ((-48, -32), (-32, -16), (-15, -5))),
 ('on', ((-18, 26), (-33, 15), (-7, 46))),
 ('off', ((-40, -22), (-38, -28), (23, 41))),
 ('on', ((-16, 35), (-41, 10), (-47, 6))),
 ('off', ((-32, -23), (11, 30), (-14, 3))),
 ('on', ((-49, -5), (-3, 45), (-29, 18))),
 ('off', ((18, 30), (-20, -8), (-3, 13))),
 ('on', ((-41, 9), (-7, 43), (-33, 15)))]

In [6]:
def single_range_intersection(r1, r2):
    # https://scicomp.stackexchange.com/a/26260
    low, high = 0, 1 # assuming in order
    if len(r1) != 2 or len(r2) != 2:
        return []
    if r2[low] > r1[high] or r1[low] > r2[high]:
        return [] # no intersection
    else:
        return (max(r1[low], r2[low]), min(r1[high], r2[high]))
    

single_range_intersection((0, 2), (2, 4)) # NOTE THIS EDGE CASE

(2, 2)

In [7]:
def range_intersection(range_1, range_2):
    return [single_range_intersection(*ranges) for ranges in zip(range_1, range_2)]

r1 = [(-82717, -58814), (9792, 34867), (31729, 41188)]
r2 = [(-62226, -36572), (-17931, -2307), (64240, 74646)]

test_range_intersections = range_intersection(r1, r2)
test_range_intersections

[(-62226, -58814), [], []]

In [8]:
def range_intersection_idxs(idx_1, idx_2):
    return [single_range_intersection(*ranges) for ranges in zip(ranges[idx_1], ranges[idx_2])]

test_range_intersections = range_intersection_idxs(0, 1)
test_range_intersections

[(-20, 26), (-21, 17), (-26, 7)]

In [9]:
def intersection_size(intersection_3d):
    # TODO: is the '+1' correct
    return prod([max(rang)-min(rang)+1 if rang else 0 for rang in intersection_3d])


# try evaluating intersections with `off` as null, empty

In [28]:
def find_total_intersection_size(onoffs, ranges, debug_print=False):

    ons = set([ii for onoff, ii in zip(onoffs, range(len(ranges))) if onoff == 'on'])
    offs = set(range(len(ranges))).difference(ons)
    
    running_total = 0

    for onoff, ii in zip(onoffs, range(len(ranges))):

        if debug_print: print(f"testing set {ii} ({onoff})") 

        # test intersections with the sets before it only
#         used_previous = set(range(ii))
        used_previous = []
        used_previous.append(set(range(ii)))

        # add current set size, if on
        if onoff == 'on':
            running_total += intersection_size(ranges[ii])
        if debug_print: print(f"\t{running_total=}")

        for way in range(2, ii+2):
            if debug_print: print(f"\ttesting {way}-way intersections")

            eligible = set(range(ii)).intersection(used_previous[-1])
            if debug_print: print(f"\teligible: {eligible}")

            even = (way % 2 == 0)

            if even:
                eligible = eligible.intersection(ons)
                if debug_print: print(f"\teligible: {eligible} (after considering only 'on')")

            used_current = set()
            
            # ugly hack instead of dealing with indexing the list of used_previous
            if even:
                used_current = used_previous[-1]
            # ugly hack instead of dealing with indexing the list of used_previous
            
            for comb in combinations(eligible, r=(way-1)):

                full_comb = tuple([ii] + list(comb)) # including ii, the current one
                if debug_print: print(f"\t\t combination {full_comb}")

                way_int = reduce(range_intersection, [ranges[idx] for idx in full_comb])

                if not [] in way_int:
                    # if a real intersection
                    used_current.update(comb)
                    size = intersection_size(way_int)

                    if even:
                        running_total -= size
                    else:
                        running_total += size

                    if debug_print: print(f"\t\t yes, {used_current=}, size: {size}, {running_total=}")

                else:
                    if debug_print: print("\t\t no")
            if debug_print: print(f"\tall intersecting with {ii}: {used_current}")
            if not used_current:
                # if no w-way intersections, definitely no w+1, ... way intersections
                if debug_print: print(f"\tNO {way}-WAY INTERSECTIONS, SKIPPING REMAINING...")
                break
            used_previous.append(used_current)
    
    return running_total

# correct_ans = 2758514936282235
correct_ans = 590784
ans = find_total_intersection_size(onoffs, ranges)
print(ans, correct_ans, correct_ans-running_total)

651264 590784 3600


# test cases

In [21]:
t1 = [('on', ((-20, 26), (-36, 17), (-47, 7))),
 ('on', ((-20, 33), (-21, 23), (-26, 28))),
 ('on', ((-22, 28), (-29, 23), (-38, 16))),]

onoffs1, ranges1 = zip(*t1)

find_total_intersection_size(onoffs1, ranges1)

225476

In [22]:

onoffs2 = ["on"]
ranges2 = [((0, 9), (0, 9), (0, 0)),
           ((0, 9), (0, 9), (0, 0)),
          ]

assert find_total_intersection_size(onoffs2, ranges2) == 100

In [23]:
onoffs3 = ["off", "off"]
ranges3 = [((0, 9), (0, 9), (0, 0)),
           ((0, 99), (0, 99), (0, 0)),
          ]
assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 0, find_total_intersection_size(onoffs3, ranges3)

testing set 0 (off)
	running_total=0
testing set 1 (off)
	running_total=0
	testing 2-way intersections
	eligible: {0}
	eligible: set() (after considering only 'on')
	all intersecting with 1: set()
	NO 2-WAY INTERSECTIONS, SKIPPING REMAINING...


In [24]:
onoffs3 = ["off", "off", "off"]
ranges3 = [((0, 9), (0, 9), (0, 0)),
           ((0, 99), (0, 99), (0, 0)),
           ((0, 999), (0, 999), (0, 0)),
          ]
assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 0, find_total_intersection_size(onoffs3, ranges3)

testing set 0 (off)
	running_total=0
testing set 1 (off)
	running_total=0
	testing 2-way intersections
	eligible: {0}
	eligible: set() (after considering only 'on')
	all intersecting with 1: set()
	NO 2-WAY INTERSECTIONS, SKIPPING REMAINING...
testing set 2 (off)
	running_total=0
	testing 2-way intersections
	eligible: {0, 1}
	eligible: set() (after considering only 'on')
	all intersecting with 2: set()
	NO 2-WAY INTERSECTIONS, SKIPPING REMAINING...


In [25]:

onoffs3 = ["on", "off"]
ranges3 = [((0, 9), (0, 9), (0, 0)),
           ((0, 99), (0, 99), (0, 0)),
          ]
assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 0, find_total_intersection_size(onoffs3, ranges3)


testing set 0 (on)
	running_total=100
testing set 1 (off)
	running_total=100
	testing 2-way intersections
	eligible: {0}
	eligible: {0} (after considering only 'on')
		 combination (1, 0)
		 yes, used_current={0}, size: 100, running_total=0
	all intersecting with 1: {0}


In [26]:

onoffs3 = ["off", "on"]
ranges3 = [((0, 9), (0, 9), (0, 0)),
           ((0, 99), (0, 99), (0, 0)),
          ]

assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 10000, find_total_intersection_size(onoffs3, ranges3)



testing set 0 (off)
	running_total=0
testing set 1 (on)
	running_total=10000
	testing 2-way intersections
	eligible: {0}
	eligible: set() (after considering only 'on')
	all intersecting with 1: set()
	NO 2-WAY INTERSECTIONS, SKIPPING REMAINING...


In [30]:
# test that all-off regions don't subtract from later overlapping 'on' regions 

onoffs3 = ["on", "off", "on"]
ranges3 = [((0, 1), (0, 1), (0, 0)),
           ((0, 2), (0, 2), (0, 0)),
           ((0, 9), (0, 9), (0, 0)),
          ]
assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 100, find_total_intersection_size(onoffs3, ranges3)


testing set 0 (on)
	running_total=4
testing set 1 (off)
	running_total=4
	testing 2-way intersections
	eligible: {0}
	eligible: {0} (after considering only 'on')
		 combination (1, 0)
		 yes, used_current={0}, size: 4, running_total=0
	all intersecting with 1: {0}
testing set 2 (on)
	running_total=100
	testing 2-way intersections
	eligible: {0, 1}
	eligible: {0} (after considering only 'on')
		 combination (2, 0)
		 yes, used_current={0, 1}, size: 4, running_total=96
	all intersecting with 2: {0, 1}
	testing 3-way intersections
	eligible: {0, 1}
		 combination (2, 0, 1)
		 yes, used_current={0, 1}, size: 4, running_total=100
	all intersecting with 2: {0, 1}


In [31]:
# test that all-off regions don't subtract from later overlapping 'on' regions 

onoffs3 = ["on", "off", "on", "off", "on"]
ranges3 = [((0, 1), (0, 1), (0, 0)),
           ((0, 2), (0, 2), (0, 0)),
           ((0, 3), (0, 3), (0, 0)),
           ((0, 4), (0, 4), (0, 0)),
           ((0, 9), (0, 9), (0, 0)),
          ]
assert find_total_intersection_size(onoffs3, ranges3, debug_print=True) == 100, find_total_intersection_size(onoffs3, ranges3)


testing set 0 (on)
	running_total=4
testing set 1 (off)
	running_total=4
	testing 2-way intersections
	eligible: {0}
	eligible: {0} (after considering only 'on')
		 combination (1, 0)
		 yes, used_current={0}, size: 4, running_total=0
	all intersecting with 1: {0}
testing set 2 (on)
	running_total=16
	testing 2-way intersections
	eligible: {0, 1}
	eligible: {0} (after considering only 'on')
		 combination (2, 0)
		 yes, used_current={0, 1}, size: 4, running_total=12
	all intersecting with 2: {0, 1}
	testing 3-way intersections
	eligible: {0, 1}
		 combination (2, 0, 1)
		 yes, used_current={0, 1}, size: 4, running_total=16
	all intersecting with 2: {0, 1}
testing set 3 (off)
	running_total=16
	testing 2-way intersections
	eligible: {0, 1, 2}
	eligible: {0, 2} (after considering only 'on')
		 combination (3, 0)
		 yes, used_current={0, 1, 2}, size: 4, running_total=12
		 combination (3, 2)
		 yes, used_current={0, 1, 2}, size: 16, running_total=-4
	all intersecting with 3: {0, 1, 2}
	te

AssertionError: 143